In [ ]:
#@markdown ## **Setup**
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider
import requests
#Creates a text document using the Raw github url. This text document will be used as a .py file for imports
def import_text(text):
  url = 'https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Exercises/assign_operators/files/%s' % text
  resp = requests.get(url)
  with open(text, 'wb') as f:
    f.write(resp.content)


import_text("frq_assign.py")




from frq_assign import *


### Representing Gate Level Logic


##Defining Signals

In SystemVerilog you need to set the size of type and size. The most common signal type is `logic`.

For example, we need to define an input, with a length of 4 bits. `input logic [3:0] signal_name`

Types of Signals:
* `input`: This represents a signal coming from an outside source, i.e., a signal from a button or the `output` of a different module. You cannot directly change an `input` inside the module.
* `output`: This represents a signal you are sending to an outside source. For example, to send to an LED or a different module. You must assign all `output` signals with a value.
* `logic`: All signals you will create in this lab will be of the type `logic`. This is the most standard data type in SystemVerilog. This is not a data type in Verilog.

When defining a new signal, there are the following parameters: `[width]'[type][value]`

`width` is a numerical value of the number of digits in the signal.

`type` is a letter specifying the base as shown below.

`value` is the representation of whatever value the signal will hold according to the appropriate base, also shown below.
* Binary (Base 2): `b`, width `4`, `4'b10001` = `17`
* Decimal (Base 10): `d`, width `2`, `2'd17` or `17` = `17`
* Octal (Base 8): `o`, width `2`, `2'o21` = `17`
* Hex (Base 16): `h`, width `2`, `2'h11` = `17`


##RTL

SystemVerilog is considered an RTL (Register Transfer Logic) programming language. Unlike most other programming languages, which are compiled into machine code, SystemVerilog is transformed into physical hardware. This means we must code it differently. For the most part, SystemVerilog isn't run line by line, all lines will execute simultaneously.

There are 3 types of signals: Inputs, Outputs and everything else.

* Inputs: Usually mapped to a physical component going into the module (e.g. Switches)
* Outputs: Usually mapped to a physical component leaving the module (e.g. LEDs)
* Signals: All of your other variables that connect the inside of the module

In this lab, your inputs will be switches, and outputs will be LEDs. 

##Assign blocks:

The simplest way to assign variables to values is using an `assign` statement.

NOTE: Assigning multiple values to the same signal will cause an error because all statements are evaluated at the same time. It's like trying to plug in multiple cords to the same USB slot. It just doesn't work.

The key word `assign` is used. The statement afterwards is constantly evaluated.   
Example:
* `assign a = b & c`;  

An Assign block only works with a single line.

#Binary Operators

## Bitwise
Bitwise Operators take 2 binary numbers then performs an operation on each pair of bits. It takes the first bits from each operand and does something, then the second set of bits and so on.

* `&` : The equivalent of AND. This operator will AND every bit in the input with the other input starting from left to right and return either a `0` or a `1`. Remember, AND will only return `1` if both bits are `1`. (e.g. `1011&0110` -> `0010`)
* `|` : The equivalent of OR. This operator will OR every bit in the input with the other input starting from left to right and return either a `0` or a `1`. Remember, OR will return `1` if either or both bits is `1`. (e.g. `1001|0011` -> `1011`)
* `~` : The equivalent of NOT. This operator will invert every bit of the given input. (e.g. `~1011` -> `0100`)
* `^` : The equivalent of XOR, or exculsive or. This operator will XOR every bit of the inputs and return a `0` if there is an even number of `1`'s or a `1` if there is an odd number of `1`'s. (e.g. `1000^1101` -> `0101`)

## Logical
This takes two operands and performs the operation on both. It should be used for just single bit wide signals. If it is given a value greater than `1` (`10` or `111`), it will treat it as `1`. 
* `&&`: ANDs the two values (e.g. `101&&100`-> `1`, `011&&000` -> `0`)
* `||`: ORs the two values (e.g. `110||000` -> `1`, `000&&000` -> `0`)
* `^^`: XORs the two values (e.g. `101^^010` -> `0`, `01^^00` -> `1`)

## Reduction
Reduction operators convert a single string of bits into a single bit. This single bit is the result of performing the operator on every other bit in the string.<br>
Reductions only take one argument.  
You can think of it as creating a gate with many single bit inputs.

* `&`: ANDs every bit. (e.g. `&1011`-> `0`, `&1111`-> `1`)
* `|`: ORs every bit. (e.g. `|1000` -> `1`)
* `^`: XORs the string. Returns `1` if there is an odd number of `1`'s. Otherwise returns `0`. (e.g. `^1101` -> `1`, `^0110` -> `0`)
* `!`: NOTs the whole string. If the input is `1` or greater, returns `0`, if it is `0`, returns `1`. (e.g. `!1011` -> `0`, `!0000` -> `1`)

##Slice
A single bit or a section of bits from an input signal of a larger size is called a slice. `signal_name[X:Y]` where X is the starting bit and Y is the ending bit.  
Examples:
* `A = 4'b0001`, `A[0]` -> `1`
* `A[1]` -> `0`, `A[3:1]` -> `000`

Note that indices start on the right side at 0 and that it is inclusive.

##Bit Shift

The `<<` shifts bits to the left, `>>` to the right. The argument on the left is what you want shifted, the argument on the right is how many bits are too be shifted. This function does not change the size of the bit string, so any bits that get cut off are gone forever. In addition, new bits are filled with `0`'s.

Examples:
- `4'b1010 << 2` -> `4'b1000`
- `4'b1111 >> 3` -> `4'b0001`
- `8'b10101010 << 3` -> `8'b01010000`

This is the easiest way for computers to multiply or divide by powers of two.

##Concatenate
If you want to combine two different signals into one, then you use concatenation `{ }`. Signals are separated by a `,`. The length of the new string is the sum of the lengths inside the `{ }`.

Example:
* `{2'b10, 2'b01}` -> `4'b1001`

##Replicate  
You can designate for certain signals to be duplicated using the `{X{Y}}` X is the number of repititions and Y is the signal or value.

Examples:  
* `{4{1'b1}}` -> `4'b1111` 
* If `a = 2'b01`, then `{{2{2'b11}}, {2{a}}}` -> `8'b11110101`
* If `b = 2'b10`, then `{{2{2'b10}}, {2{b[0]}}}` -> `6'b101000`

When using both replicate and concatenate in a single statement, make sure that replicate gets its own set of brackets for each time it is used.

# Practice Questions

In [ ]:
#@markdown Q1
print_frq_grid(1)

In [ ]:
#@markdown Q2
print_frq_grid(2)

In [ ]:
#@markdown Q3
print_frq_grid(3)

In [ ]:
#@markdown Q4
print_frq_grid(4)

In [ ]:
#@markdown Q5
print_frq_grid(5)

In [ ]:
#@markdown Q6
print_frq_grid(6)

In [ ]:
#@markdown Q7
print_frq_grid(7)

In [ ]:
#@markdown Q8
print_frq_grid(8)

In [ ]:
#@markdown Q9
print_frq_grid(9)

In [ ]:
#@markdown Q10
print_frq_grid(10)

# Multiline Operations

## Always Comb Blocks:

What if we want to use If Else branches or multiple lines without using Assign over and over? We can use an `always_comb` block. This turns everything inside into combinational logic.

Always_comb blocks are contained in `begin` and `end` keywords. 
The keywords `begin` and `end` are also needed when other statements are more than one line long.  

```
always_comb begin
  ...
  ...
end
```

## If/Else Branches:

We can use if else branches inside an always_comb block.

NOTE: If you want to have multiple lines after your `if`, `else if` or `else` statement, add `begin` and `end`.

These are formed like just like in other programming languages.
```
if (a == 1) begin
  ...
end
else if (a == 0) begin
  ...
end
else begin
  ...
end
```

## Latches

One thing you must be careful about is latches. We won't discuss what a latch is here but we will discuss why they are a problem. Think about a circuit made of gates. In an If/Else block, only one set of statements will be active at a time, the `if` or the `else`, never both.

Suppose we have a two bit switch called `sw`. When only the right bit is high, we want the signal `a` to be high. When only the left bit is high, we want `b` to be high. Otherwise, we want `c` to be set to high. You might think an easy solution would be:
```
always_comb begin
    if(sw == 2'b01) a=1'b1;
    else if(sw == 2'b10) b=1'b1;
    else c=1'b1;
end
```
### BUT YOU WOULD BE WRONG.

The above will actually result in a latch, which is bad. We don't know what would happen to any of the other signals from just one branch. Remember, SystemVerilog is supposed to represent a physical circuit. All possible signals are going to be connected one way or the other. The signals `a`, `b`, and `c` are connected to the rest of the circuit even if one doesn't really get used. 

In the first case, when only the right bit is high, the circuit won't know what to do with both `b` and `c` and so the compiler will infer a latch and fail to compile. There are two easy ways to avoid this:
* The easiest way is to default all signals to low at the beginning of the `always_comb` block. Any line giving a signal after the default will override it. This method has the least amount of code.
```
always_comb begin
    a = 1'b0;
    b = 1'b0;
    c = 1'b0;
    if(sw == 2'b01) a = 1'b1;
    else if(sw == 2'b10) b = 1'b1;
    else c = 1'b1;
end
```
* The other way is to make sure that every signal is set to something in every branch. This method is prone to error as it is easy to forget a single signal in a single branch.
```
always_comb begin
    if(sw == 2'b01)begin
        a = 1'b1;
        b = 1'b0;
        c = 1'b0;
    end
    else if(sw == 2'b10) begin
        a = 1'b0;
        b = 1'b1;
        c = 1'b0;    
    end
    else begin 
        a = 1'b0;
        b = 1'b0;
        c = 1'b1;    
    end
end
```